# Cell Tower Coverage

This notebook aims to emulate the [Gurobi Cell Tower Coverage example](https://github.com/Gurobi/modeling-examples/blob/master/cell_tower_coverage/cell_tower.ipynb) using OptVerse.
This example demonstrates solving a cell tower placement problem to provide signal coverage to the largest number of people possible.

## Prerequisites

The repo containing this, and other examples is available [through CodeHub](link)

## Problem Description

The following section is taken verbatim from [Gurobi Cell Tower Coverage example](https://github.com/Gurobi/modeling-examples/blob/master/cell_tower_coverage/cell_tower.ipynb).

A telecom company needs to build a set of cell towers to provide signal coverage for the inhabitants of a given city. A number of potential locations where the towers could be built have been identified. The towers have a fixed range, and -due to budget constraints- only a limited number of them can be built. Given these restrictions, the company wishes to provide coverage to the largest percentage of the population possible. To simplify the problem, the company has split the area it wishes to cover into a set of regions, each of which has a known population. The goal is then to choose which of the potential locations the company should build cell towers on -in order to provide coverage to as many people as possible.

The Cell Tower Coverage Problem is an instance of the Maximal Covering Location Problem. It is also related to the Set Cover Problem.

## Solution Approach

The following section is taken verbatim from [Gurobi Cell Tower Coverage example](https://github.com/Gurobi/modeling-examples/blob/master/cell_tower_coverage/cell_tower.ipynb).

Mathematical programming is a declarative approach where the modeler formulates a mathematical optimization model that captures the key aspects of a complex decision problem. The OptVerse Optimizer solves such models using state-of-the-art mathematics and computer science.

A mathematical optimization model has five components, namely:

* Sets and indices.
* Parameters.
* Decision variables.
* Objective function(s).
* Constraints.

We now present a mixed-integer programming (MIP) formulation for the Cell Tower Coverage Problem.

## Model Formulation

### Sets and Indices

$i \in T$: Index and set of potential sites to build a tower.

$j \in R$: Index and set of regions.

$G(T,R,E)$: A bipartite graph defined over the set $T$ of potential sites to build a tower, the set of regions $R$ that we want to cover, and $E$ is the set of edges, where we have an edge $(i,j) \in E$ if region $j \in R$ can be covered by a tower on location $i \in T$.

### Parameters

$c_{i} \in \mathbb{R}^+$: The cost of setting up a tower at site $i$.

$p_{j} \in \mathbb{N}$: The population at region $j$.

### Decision Variables

$covered_{j} \in \{0, 1 \}$: This variable is equal to 1 if region $j$ is covered; and 0 otherwise.

$build_{i} \in \{0, 1 \}$: This variable is equal to 1 if tower $i$ is built; and 0 otherwise.

### Objective Function

- **Population covered**. We seek to maximize the total population covered by the towers.

$$\text{Max} \quad Z = \sum_{j \in R} p_{j} \cdot covered_{j}$$

### Constraints

- **Coverage**. For each region $j \in R$ ensure that at least one tower that covers a region must be selected.

$$\sum_{(i,j) \in E} build_{i} \geq covered_{j} \quad \forall j \in R$$

- **Budget**. We need to ensure that the total cost of building towers do not exceed the allocated budget.

$$\sum_{i \in T} c_{i} \cdot build_{i} \leq \text{budget}$$

## Python Implementation

### Example
The following section is taken verbatim from [Gurobi Cell Tower Coverage example](https://github.com/Gurobi/modeling-examples/blob/master/cell_tower_coverage/cell_tower.ipynb).

This example considers a bipartite graph for 6 towers and 9 regions. The following table illustrates which regions (columns) are covered by each cell tower site (rows).

| <i></i> | Region 0 | Region 1 | Region 2 | Region 3 | Region 4 | Region 5 | Region 6 | Region 7 | Region 8 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |  --- |
| Tower 0 | 1 | 1 | - | - | - | 1 | - | - |  - |
| Tower 1 | 1 | - | - | - | - | - | - | 1 |  1 |
| Tower 2 | - | - | 1 | 1 | 1 | - | 1 | - |  - |
| Tower 3 | - | - | 1 | - | - | 1 | 1 | - |  - |
| Tower 4 | 1 | - | 1 | - | - | - | 1 | 1 |  1 |
| Tower 5 | - | - | - | 1 | 1 | - | - | - |  1 |

The population at each region is stated in the following table.

| <i></i> | Region 0 | Region 1 | Region 2 | Region 3 | Region 4 | Region 5 | Region 6 | Region 7 | Region 8 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Population | 523 | 690 | 420 | 1010 | 1200 | 850 | 400 | 1008 | 950 |

The cost to build a cell tower at each location site is stated inthe following table.

| <i></i> | Cost (millions of USD) |
| --- | --- |
| Tower 0 | 4.2 |
| Tower 1 | 6.1 |
| Tower 2 | 5.2 |
| Tower 3 | 5.5 |
| Tower 4 | 4.8 |
| Tower 5 | 9.2 | 

The allocated budget is $\$20,000,000$.

We will now illustrate how to solve the problem described through OptVerse.

### Environment Configuration

To use the OptVerse Python module the user must import the package.

In [ ]:
from optvpy import *

### Data Preparation

We will first prepare the data required to construct the model.
For pedagogical purposes, we will just construct this data explicitly.
This example considers a bipartite graph for 6 towers and 9 regions.

In [ ]:
# Parameters
budget = 20
regions = list(range(9))
population = {0: 523, 1: 690, 2: 420, 3: 1010, 4: 1200, 5: 850, 6: 400, 7: 1008, 8: 950}

sites = list(range(6))
coverage = {
    0: {0, 1, 5},
    1: {0, 7, 8},
    2: {2, 3, 4, 6},
    3: {2, 5, 6},
    4: {0, 2, 6, 7, 8},
    5: {3, 4, 8}
}
cost = {0: 4.2, 1: 6.1, 2: 5.2, 3: 5.5, 4: 4.8, 5: 9.2}

### Model Creation

To create a model, we must first instantiate an environment, and pass it to the constructor for our model object.

In [ ]:
env = OPTVEnv()
model = OPTVModel(env)

### Add Variables

Next, we add our 'build' and 'is_covered' decision variables to our model.
We set the linear coefficients to the objective through the variables addition interface.

In [ ]:
# Decision variables
build = m.AddVars(sites, lb=0, ub=1, vtype=BINARY, name="Build")
is_covered = m.AddVars(regions, lb=0, ub=1, vtype=BINARY, name="Is_covered")

### Add Constraints

Now we add the constraints to our model.

In [ ]:
# Constraints
m.AddConstrs((sum(build[t] for t in sites if r in coverage[t]) >= is_covered[r]
              for r in regions), name="Build2cover")

m.AddConstr(sum(cost[s] * build[s] for s in sites) <= budget, name="budget")

### Set Objective

Set the objective to maximize population coverage.

In [ ]:
# Objective
m.setObjective(sum(population[r] * is_covered[r] for r in regions), OPTVSense.MAXIMIZE)

### Optimize the Model

Finally, we can run the optimization on the problem.

In [ ]:
# Optimize
m.optimize()

## Analyze the Solution

The result of the optimization model shows the maximum population that can be covered with the $20,000,000 budget.
Let's see the solution that achieves that optimal result.

This plan determines at which site locations to build a cell tower.

In [ ]:
for tower in build:
    if (abs(tower.Get(OPTVDblAttr.X)) > 1e-6):
        print(f"\n Build a cell tower at location Tower {tower.Index()}.")

### Coverage Plan

This plan determines which regions are covered by the towers built.

In [ ]:
for i in range(num_regions):
    if (abs(is_covered[i].Get(OPTVDblAttr.X)) > 1e-6):
        print(f"\n Region {i} with population {population[i]} is covered.")

### Performance Metrics

Calculate coverage and budget utilization metrics.

In [ ]:
# Calculate total population covered
total_population = sum(population)
total_covered = 0

for i in range(num_regions):
    if (abs(is_covered[i].Get(OPTVDblAttr.X)) > 1e-6):
        total_covered += population[i]

coverage_percentage = round(100 * total_covered / total_population, 2)
print(f"\n The population coverage associated to the cell towers build plan is: {coverage_percentage} %")

# Calculate budget consumption
total_cost = 0
for tower in build:
    if (abs(tower.Get(OPTVDblAttr.X)) > 1e-6):
        total_cost += cost[tower.Index()]

budget_consumption = round(100 * total_cost / budget, 2)
print(f"\n The percentage of budget consumed associated to the cell towers build plan is: {budget_consumption} %")

## Conclusion

In this example, we addressed a cell tower coverage problem where we want to build cell towers to provide signal coverage to the largest number of people possible while satisfying a budget constraint.
We learned how to formulate the problem as a MIP model.
Also, we learned how to implement the MIP model formulation and solve it using the OptVerse Python API.